In [1]:
import numpy as np
from itertools import compress
from scipy.spatial import distance_matrix
import time
import pandas as pd

In [2]:
class HPFold:

    def __init__ (self, s):
        legal = {'h':'h', 'p':'p', 'H':'h', 'P':'p'}
        self.seq = []
        i = 1
        for c in s:
            if c in legal.keys():
                if legal[c] == 'h' and i % 2 == 0:
                    self.seq.append('H')
                else:
                    self.seq.append(legal[c])
                i = i + 1

    def __len__ (self):
        return len(self.seq)
        
    def SetRelFold (self, relfold):
        """
        Fold seq according to a description in relavtive format, i.e.
        a sequence of {f,l,r}'s which describe each step as (f)orward,
        (l)eft, or (r)ight.
        """
        turn = {'f':0, 'l':-1, 'r':1}
        direction = {0:'e', 1:'s', 2:'w', 3:'n'}
        absfold = []
        curr = 0
        for relstep in relfold:
            absstep = (curr + turn[relstep]) % 4
            absfold.append(direction[absstep])
            curr = absstep
        return self.SetAbsFold(absfold)

    def SetAbsFold (self, absfold):
        """
        Fold seq according to a description in absolute format, i.e.
        s sequence of {n,s,e,w}'s which describe each step as (n)orth,
        (s)outh, (e)ast, or (w)est.
        """
        self.legal_fold = (True, 0)
        self.grid = {}
        self.grid[0,0] = [0]
        i = j = self.min_i= self.max_i = self.min_j = self.max_j = 0
        k = 1
        for step in absfold:
            if step == 'n':
                i = i - 1
            elif step == 's':
                i = i + 1
            elif step == 'e':
                j = j + 1
            elif step == 'w':
                j = j - 1
            if (i,j) in self.grid.keys():
                self.legal_fold = (False, k)
                self.grid[i,j].append(k)
            else:
                self.grid[i,j] = [k]
            k = k + 1
            self.min_i = min(i, self.min_i)
            self.max_i = max(i, self.max_i)
            self.min_j = min(j, self.min_j)
            self.max_j = max(j, self.max_j)                        
        return self.legal_fold[0]

    def ContainNeighbors(self, l1, l2):
        """
        Returns true if there exists k1 in l1 and k2 in l2 such that
        abs(k1-k2) is 1, i.e. if the indices in l1 and l2 contain a
        pair of neighbors in seq.
        """
        res = False
        for k1 in l1:
            for k2 in l2:
                if abs(k1-k2) == 1:
                    res = True
        return res

    def ContainHHs(self, l1, l2):
        """
        Returns true if there exists k1 in l1 and k2 in l2 where there
        is a 'h' at position k1 and k2 in seq, i.e. if the indices in
        l1 and l2 contain a pair which can make a h-h bond.
        """
        res = False
        for k1 in l1:
            for k2 in l2:
                if (self.seq[k1] == "h" or self.seq[k1] == "H") and (self.seq[k2] == "h" or self.seq[k2] == "H"):
                    res = True
        return res
        
    def PrintFold (self):
        """
        Print fold and output its score
        """
        score = 0
        print()
        for i in range(self.min_i, self.max_i+1):
            for j in range(self.min_j, self.max_j+1):
                if (i,j) in self.grid.keys():
                    l1 = self.grid[i,j]
                    if len(l1) == 1:
                        print(self.seq[l1[0]], end="")
                    else:
                        print("X", end="")
                    if (i,j+1) in self.grid.keys():
                        l2 = self.grid[i,j+1]
                        if self.ContainNeighbors(l1,l2):
                            print("-", end="")
                        elif self.ContainHHs(l1, l2):
                            print("*", end="")
                            score = score + 1
                        else:
                            print(" ", end="")
                    else:
                        print(" ", end="")
                else:
                    print(".", end="")
                    print(" ", end="")
            print()

            for j in range(self.min_j, self.max_j+1):
                if (i,j) in self.grid.keys() and (i+1,j) in self.grid.keys():
                    l1 = self.grid[i,j]
                    l2 = self.grid[i+1,j]
                    if self.ContainNeighbors(l1,l2):
                        print("|", end="")
                    elif self.ContainHHs(l1,l2):
                        print("*", end="")
                        score = score + 1
                    else:
                        print(" ", end="")
                else:
                    print(" ", end="")
                print(" ", end="")
            print()

        if self.legal_fold[0]:
            print("Score: %d" % (score))
            return score
        else:
            print("Illegal fold after %d steps" % (self.legal_fold[1]))


def make_absfold(f):

    absfold = []
    for c in f.lower():
        if c == 'n' or c == 's' or c == 'e' or c == 'w':
            absfold.append(c)
    return absfold

def make_relfold(f):

    relfold = []
    for c in f.lower():
        if c == 'f' or c == 'l' or c == 'r':
            relfold.append(c)
    return relfold


def print_usage():
    print("""
Usage:

hpview.py <seq> <fold>

Displays  folding <fold> of sequence  <seq> in the  2D HP model, where
<seq> is a string over the  alphabet {h,p} and  <fold> is a folding of
<seq> in either relative or absolute format. The length of <fold> must
be exactly one less than the length of <seq>.

A folding in    relative format is   a sequence  over  {f,l,r},  which
describes  the fold  as a sequence  of  steps  (f)orward, (l)left,  or
(r)ight.

A folding  in  absolute format  is  a  sequence  over {n,s,e,w}, which
describes the  fold as a sequence of   steps to the  (n)orth, (s)outh,
(e)east, or (w)est.              
""")

In [3]:
def find_energy(sequence):
    
    seq_len = len(sequence)

    combis = list()

    for i in range(2, seq_len//2):
        string = list()
        for j in range(i):
            remainder = j % i
            if remainder == 0:
                string.append("n"+"e"*(i-1))
            if remainder == 1:
                string.append("n"+"w"*(i-1))
        seq = "".join(string)
        seq1 = seq*(seq_len//len(seq))
        seq = seq1 + seq[:seq_len%len(seq)]
        combis.append(seq[:-1])
    
    
    scores = list()
    for i in range(len(combis)):
        start_time = time.time()
        seq = HPFold(sequence)
        fold  = combis[i]
        print(fold)
        absfold = make_absfold(fold)
        relfold = make_relfold(fold)
        if len(absfold) == len(seq) - 1:
             seq.SetAbsFold(absfold)
        elif len(relfold) == len(seq) - 1:
            seq.SetRelFold(relfold)
        end_time = time.time()
        score = seq.PrintFold()
        scores.append(score)
    
    combis[np.argmax(scores)]
    
    return combis[np.argmax(scores)]

In [4]:
timing = list()
scores = list()
length = list()
optimum = [4, 8, 9, 9, 10, 9, 8, 14, 23, 21, 36, 42, 53, 48, 50]


sequences = ["hhppppphhppphppphp", 
             "hphphhhppphhhhpphh", 
             "phpphphhhphhphhhhh", 
             "hphpphhphpphphhpphph", 
             "hhhpphphphpphphphpph",
             "hhpphpphpphpphpphpphpphh",
             "pphpphhpppphhpppphhpppphh",
             "ppphhpphhppppphhhhhhhpphhpppphhpphpp",
             "pphpphhpphhppppphhhhhhhhhhpppppphhpphhpphpphhhhh",
             "hhphphphphhhhphppphppphpppphppphppphphhhhphphphphh",
             "pphhhphhhhhhhhppphhhhhhhhhhphppphhhhhhhhhhhhpppphhhhhhphhphp",
             "hhhhhhhhhhhhphphpphhpphhpphpphhpphhpphpphhpphhpphphphhhhhhhhhhhh",
             "hhhhpppphhhhhhhhhhhhpppppphhhhhhhhhhhhppphhhhhhhhhhhhppphhhhhhhhhhhhppphpphhpphhpphph",
             "pppppphphhppppphhhphhhhhphhpppphhpphhphhhhhphhhhhhhhhhphhphhhhhhhppppppppppphhhhhhhpphphhhpppppphphh",
             "ppphhpphhhhpphhhphhphhphhhhpppppppphhhhhhpphhhhhhppppppppphphhphhhhhhhhhhhpphhhphhphpphphhhpppppphhh"]

for sequence in sequences:
    fold = find_energy(sequence)
    
    start_time = time.time()
    seq = HPFold(sequence)
    absfold = make_absfold(fold)
    relfold = make_relfold(fold)
    if len(absfold) == len(seq) - 1:
         seq.SetAbsFold(absfold)
    # elif len(relfold) == len(seq) - 1:
    #     seq.SetRelFold(relfold)
    end_time = time.time()
    print(fold)
    
    score = seq.PrintFold()
    scores.append(score)
    length.append(len(sequence))
    timing.append(end_time-start_time)

nenwnenwnenwnenwn

p . 
|   
h-p 
  | 
p-p 
|   
h-p 
  | 
p-p 
|   
h-H 
  | 
p-p 
|   
p-p 
  | 
H-p 
|   
h . 
    
Score: 0
neenwwneenwwneenw

. p-h 
    | 
p-p-p 
|     
h-p-p 
*   | 
H-h-p 
|     
p-p-p 
    | 
H-p-p 
|     
h . . 
      
Score: 1
neeenwwwneeenwwwn

p . . . 
|       
h-p-p-p 
      | 
p-p-p-h 
|       
h-H-p-p 
*     | 
H-p-p-p 
|       
h . . . 
        
Score: 1
neeeenwwwwneeeenw

. . . p-h 
        | 
p-h-p-p-p 
|         
p-p-h-H-p 
        | 
H-p-p-p-p 
|         
h . . . . 
          
Score: 0
neeeeenwwwwwneeee

p-p-p-h-p . 
|           
h-p-p-p-h-H 
*         | 
H-p-p-p-p-p 
|           
h . . . . . 
            
Score: 1
neeeeeenwwwwwwnee

p-h-p . . . . 
|             
p-p-h-p-p-p-h 
            | 
H-p-p-p-p-p-H 
|             
h . . . . . . 
              
Score: 0
neeeeeeenwwwwwwwn

p . . . . . . . 
|               
h-p-p-p-h-p-p-p 
*             | 
H-p-p-p-p-p-H-h 
|               
h . . . . . . . 
                
Score: 1
neenwwneenwwneenw

. p-h 
  

              
Score: 2
neeeeeeenwwwwwwwneeeeeee

H-h-p-p-p-p-H-h 
|               
p-p-p-p-h-H-p-p 
        * *   | 
p-h-p-p-H-h-p-p 
|               
p . . . . . . . 
                
Score: 2
neeeeeeeenwwwwwwwwneeeee

p-p-p-p-H-h . . . 
|                 
h-H-p-p-p-p-h-H-p 
  *             | 
p-h-p-p-H-h-p-p-p 
|                 
p . . . . . . . . 
                  
Score: 1
neeeeeeeeenwwwwwwwwwneee

p-p-H-h . . . . . . 
|   * *             
p-p-h-H-p-p-p-p-h-H 
                  | 
p-h-p-p-H-h-p-p-p-p 
|                   
p . . . . . . . . . 
                    
Score: 2
neeeeeeeeeenwwwwwwwwwwne

H-h . . . . . . . . . 
|                     
p-p-p-p-h-H-p-p-p-p-h 
        * *         | 
p-h-p-p-H-h-p-p-p-p-H 
|                     
p . . . . . . . . . . 
                      
Score: 2
neeenwwwneeenwwwneeenwww

h-H-p-p 
* *   | 
H-h-p-p 
|       
p-p-p-p 
      | 
p-p-H-h 
|   * * 
p-p-h-H 
      | 
p-h-p-p 
|       
p . . . 
        
Score: 4
nenwnenwnenwnenwnenwnenwnenwnenwnen

        * *     * *       | 
p-h-p-p-H-h-p-p-H-h-p-p-p-p 
|                           
p . . . . . . . . . . . . . 
                            
Score: 10
neeeeeeeeeeeeeenwwwwwwwwwwwwwwneeeeeeeeeeeeeenw

. . . . . . . . . . . . . H-h 
                          * | 
p-h-H-p-p-h-H-p-p-h-p-p-H-h-H 
|         * *     *     * * * 
p-p-p-p-p-H-h-H-h-H-h-H-h-H-h 
          *     * *         | 
p-h-p-p-H-h-p-p-H-h-p-p-p-p-p 
|                             
p . . . . . . . . . . . . . . 
                              
Score: 10
neeeeeeeeeeeeeeenwwwwwwwwwwwwwwwneeeeeeeeeeeeee

H-p-p-h-H-p-p-h-p-p-H-h-H-h-H . 
|             *     * * * * *   
h-p-p-p-p-p-p-H-h-H-h-H-h-H-h-H 
                * *           | 
p-h-p-p-H-h-p-p-H-h-p-p-p-p-p-h 
|                               
p . . . . . . . . . . . . . . . 
                                
Score: 8
neeeeeeeeeeeeeeeenwwwwwwwwwwwwwwwwneeeeeeeeeeee

p-h-H-p-p-h-p-p-H-h-H-h-H . . . . 
| * *             * * * *         
p-H-h-p-p-p-p-p-p-H-h-H-h-H-h-H-h 


. . . . . . . . . . . . . . . H 
                              | 
p-p-H-p-H-h-H-h-p-h-p-h-p-h-p-h 
|         *                     
p-H-p-p-p-H-p-p-p-p-h-p-p-p-h-p 
                    *         | 
H-p-H-p-H-p-H-p-H-h-H-h-p-h-p-p 
|                               
h . . . . . . . . . . . . . . . 
                                
Score: 2
neeeeeeeeeeeeeeeenwwwwwwwwwwwwwwwwneeeeeeeeeeeeee

H-p-H-h-H-h-p-h-p-h-p-h-p-h-H . . 
|     *       *                   
p-p-p-H-p-p-p-H-p-p-p-p-h-p-p-p-h 
                                | 
H-p-H-p-H-p-H-p-H-h-H-h-p-h-p-p-p 
|                                 
h . . . . . . . . . . . . . . . . 
                                  
Score: 2
neeeeeeeeeeeeeeeeenwwwwwwwwwwwwwwwwwneeeeeeeeeeee

H-h-H-h-p-h-p-h-p-h-p-h-H . . . . . 
| *       *       *                 
p-H-p-p-p-H-p-p-p-H-p-p-p-p-h-p-p-p 
                  *               | 
H-p-H-p-H-p-H-p-H-h-H-h-p-h-p-p-p-h 
|                                   
h . . . . . . . . . . . . . . . . . 
           

h-p-h-H-h-H-h-H-h-H-h-H-p-p 
    * *   * * * * * * *   | 
p-h-H-h-p-h-H-h-H-h-H-h-H-p 
|                           
p . . . . . . . . . . . . . 
                            
Score: 25
neeeeeeeeeeeeeenwwwwwwwwwwwwwwneeeeeeeeeeeeeenwwwwwwwwwwwww

. p-h-p-h-H-p-H-h-H-h-H-h-p-p 
    *   * *   * * * * * *   | 
p-h-H-h-H-h-H-h-H-h-H-h-H-p-p 
|   *   * * * * * * * * *     
p-p-h-p-h-H-h-H-h-H-h-H-h-H-p 
    *     * * * * * * * *   | 
p-h-H-h-p-h-H-h-H-h-H-h-H-p-p 
|                             
p . . . . . . . . . . . . . . 
                              
Score: 28
neeeeeeeeeeeeeeenwwwwwwwwwwwwwwwneeeeeeeeeeeeeeenwwwwwwwwww

. . . . . p-h-p-h-H-p-H-h-H-h-H 
            *   * *           | 
H-h-H-h-H-h-H-h-H-h-H-p-p-p-p-h 
|       *   * * * * *         * 
h-p-p-p-h-p-h-H-h-H-h-H-h-H-h-H 
            * * * * * * *     | 
p-h-H-h-p-h-H-h-H-h-H-h-H-p-p-p 
|                               
p . . . . . . . . . . . . . . . 
                                
Score: 17
neeeeeeeeeeeeeeeenwwwwwwwwwwwwwwww

...

|                                                                                   
p . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
                                                                                    
Score: 18
neeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeenwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwneeeeeeeeeeee

H-h-H-p-p-p-p-p-p-h-p-h-H . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
| *               *                                                                   
p-H-p-p-h-H-h-H-h-H-h-p-p-p-p-p-p-p-p-p-p-p-h-H-h-H-h-H-h-p-h-H-p-H-h-H-h-H-h-H-h-H-h 
          *   * *                           *   * *         * *     * *   * * * * * | 
p-p-p-p-p-h-p-h-H-p-p-p-p-p-H-h-H-p-H-h-H-h-H-p-H-h-p-p-p-p-H-h-p-p-H-h-p-h-H-h-H-h-p 
|                                                                                     
p . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
                         

| *   * *   
p-H-p-H-h-p 
      * * | 
H-p-p-h-H-h 
|     * * * 
h-H-h-H-h-H 
*   * * * | 
H-p-H-h-H-h 
|   *       
h-p-h-p-p-p 
          | 
p-p-p-p-p-p 
|           
h-H-h-H-h-H 
* * * *   | 
H-h-H-h-p-p 
| *         
h-H-p-p-p-p 
* *       | 
H-h-p-p-p-p 
| *         
h-H-p-H-h-p 
* *     * | 
H-h-H-p-H-h 
|   *   * * 
p-p-h-H-h-H 
    * *   | 
p-p-H-h-p-p 
|           
p . . . . . 
            
Score: 29
neeeeeenwwwwwwneeeeeenwwwwwwneeeeeenwwwwwwneeeeeenwwwwwwneeeeeenwwwwwwneeeeeenwwwwwwneeeeeenwwwwwwn

H . . . . . . 
|             
h-H-p-p-p-p-p 
  *         | 
p-h-p-h-H-h-p 
| *   * *     
p-H-p-H-h-p-h 
  *         | 
H-h-H-p-p-h-H 
| * *     * * 
h-H-h-H-h-H-h 
  *   * *   | 
p-h-p-h-H-p-H 
|             
p-p-p-p-p-p-p 
            | 
H-h-H-h-H-h-p 
|   * * * *   
p-p-h-H-h-H-h 
            | 
p-p-p-p-p-p-H 
|             
p-p-h-H-h-H-p 
    * *   * | 
H-p-H-h-p-h-H 
|         * * 
h-H-p-p-h-H-h 
            | 
p-p-H-h-p-p-H 
|             
p . . . . . . 
              
Score:

h-p-h-p-p-H-p-H-h-p-h-H-h-p-p-H-h-H-h-H-h-H 
*   *                         * *   * * * | 
H-h-H-h-p-p-p-p-p-p-p-p-p-h-p-h-H-p-H-h-H-h 
| *                                 * * * * 
h-H-p-p-h-H-h-H-h-H-p-p-p-p-p-p-p-p-h-H-h-H 
            * * * *                   * * | 
p-p-H-h-p-p-H-h-H-h-p-p-H-h-H-p-H-h-p-h-H-p 
|                                           
p . . . . . . . . . . . . . . . . . . . . . 
                                            
Score: 20
neeeeeeeeeeeeeeeeeeeeeenwwwwwwwwwwwwwwwwwwwwwwneeeeeeeeeeeeeeeeeeeeeenwwwwwwwwwwwwwwwwwwwwwwneeeeee

p-p-p-p-H-h-H . . . . . . . . . . . . . . . . 
|       *   *                                 
p-p-h-H-h-p-h-p-p-H-p-H-h-p-h-H-h-p-p-H-h-H-h 
                      *     *   *     * * * | 
H-h-p-p-p-p-p-p-p-p-p-h-p-h-H-p-H-h-H-h-H-h-H 
| *                   *                 * * * 
h-H-h-H-p-p-h-H-h-H-h-H-p-p-p-p-p-p-p-p-h-H-h 
    * *     * * * *                     *   | 
p-p-H-h-p-p-H-h-H-h-p-p-H-h-H-p-H-h-p-h-H-p-H 
|               

                                                                      
Score: 20
neeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeenwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwneeeeeeeeeeeeeeeeeeeeeeeeee

H-p-p-h-H-h-p-h-H-p-H-p-p-h-p-h-H-h-p-p-p-p-p-p-H-h-H . . . . . . . . . 
|     * * *   * *                               * * *                   
h-H-h-H-h-H-h-H-h-H-p-H-h-p-h-p-p-p-p-p-p-p-p-p-h-H-h-H-h-H-p-p-h-H-h-H 
    * *     * * * *     *   *                   * *                 * | 
p-p-H-h-p-p-H-h-H-h-p-p-H-h-H-p-H-h-p-h-H-p-H-h-H-h-p-p-p-p-p-p-p-p-H-h 
|                                                                       
p . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
                                                                        
Score: 19
neeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeenwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwneeeeeeeeeeeeeeeeeeeeeeee

p-h-H-h-p-h-H-p-H-p-p-h-p-h-H-h-p-p-p-p-p-p-H-h-H . . . . . . . . . . . . 
| * * *   * *   *     *   * *                   

                                                                                              
Score: 13
neeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeenwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwnee

H-h-H . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
|                                                                                               
p-p-p-p-p-p-h-H-h-p-h-p-p-H-p-H-h-p-h-H-h-p-p-H-h-H-h-H-h-H-h-H-h-H-p-H-h-p-h-p-p-p-p-p-p-p-p-p 
            * * *         *     *     * *     * * *                   * *   *                 | 
p-p-H-h-p-p-H-h-H-h-p-p-H-h-H-p-H-h-p-h-H-p-H-h-H-h-p-p-p-p-p-p-p-p-H-h-H-h-H-h-p-p-H-h-H-h-H-h 
|                                                                                               
p . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
                                                                                                
Score: 13
neeeeeee

In [5]:
df = pd.DataFrame({"sequence":list(range(len(sequences))),
             "length": length,
             "optimum": optimum,
             "score": scores,
             "time": timing})

df.to_csv("data_from_SMPL.csv")
df

,sequence,length,optimum,score,time
0,0,18,4,1,0.000000
1,1,18,8,5,0.000000
2,2,18,9,6,0.000000
3,3,20,9,5,0.000000
4,4,20,10,4,0.001001
5,5,24,9,4,0.000000
6,6,25,8,4,0.000000
7,7,36,14,7,0.000000
8,8,48,23,12,0.000000
9,9,50,21,11,0.000000
